Importing the required packages

In [14]:
import os

# Import the pages I need here
from astropy.table import Table
import numpy as np          # For number handling
import pandas as pd         # For table/data management
import matplotlib.pyplot    # For plotting data

The code will be sorted into two different parts.

The goal for this piece of code is:
- Clean K2 and APOKSAC Data
- Using K2 and APOKSAC Data to train models
- Use K2 model to predict the mass for APOGEE and GALAH
- Use APOKSAC model to predict mass for APOGEE

Cleaning the GALAH Data
The goal for GALAH is to get a table contain the following:
- Effective Temperature
- Log g
- Iron Abundance [Fe/H]
- Alpha Abundance [alpha/Fe]
- Nitrogen Abundance [N/Fe]
- Oxygen Abundance [O/Fe]

In [44]:
# Import the data and change into a data frame
galah = Table.read("Data/galah_dr4_allspec_220713.fits", format = "fits")
names = [name for name in galah.colnames if len(galah[name].shape) <= 1]
galah = galah[names].to_pandas()

For Apogee Data, there are a few recommendations (this usually indicates that the data has some issues, e.g. unreliably broadening and other issues) for the flag that should be applied:
- `snr_c3_iraf > 30`
- `flag_sp == 0`
- For any element `X`, the flag `flag_X_fe == 0`

In [45]:
# Filter the data
# galah = galah.loc[galah["snr_c3_iraf"] > 30]
galah = galah.loc[galah["flag_sp"] == 0]
galah = galah.loc[galah["flag_fe_h"] == 0]
galah = galah.loc[galah["flag_al_fe"] == 0]
galah = galah.loc[galah["flag_n_fe"] == 0]
galah = galah.loc[galah["flag_o_fe"] == 0]

Now the appropriate flag has been used, we extract only the useful columns

In [46]:
galah = galah[["teff", "logg", "fe_h", "al_fe", "n_fe", "o_fe"]]
galah.to_csv("Data/cleaned_galah.csv", index=False)


GALAH Data has now been cleaned
